In [1]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import random
seed = 10
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
import eli5

In [2]:
%%time
train_transaction = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

# train_pred = pd.read_csv('./simple_xgboost_cv.csv', index_col='TransactionID')
# test_pred = pd.read_csv('./simple_xgboost_pred.csv', index_col='TransactionID')
# test_pred['pred'] = (test_pred['pred1'] + test_pred['pred2'] + test_pred['pred3'] )/3
# del test_pred['pred1'],test_pred['pred2'],test_pred['pred3']

# train_f2 = pd.read_csv('../input/f2_train.csv', index_col='TransactionID')
# del train_f2['isFraud']
# test_f2 = pd.read_csv('../input/f2_test.csv', index_col='TransactionID')

sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

CPU times: user 45.3 s, sys: 4.06 s, total: 49.4 s
Wall time: 49.3 s


In [3]:
train_transaction['hour'] = train_transaction['TransactionDT'].map(lambda x:(x//3600)%24)
test_transaction['hour'] = test_transaction['TransactionDT'].map(lambda x:(x//3600)%24)
train_transaction['weekday'] = train_transaction['TransactionDT'].map(lambda x:(x//(3600 * 24))%7)
test_transaction['weekday'] = test_transaction['TransactionDT'].map(lambda x:(x//(3600 * 24))%7)

In [4]:
train_test = train_transaction.append(test_transaction)
for col in "ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,M1,M2,M3,M4,M5,M6,M7,M8,M9".split(","):
    col_count = train_test.groupby(col)['TransactionDT'].count()
    train_transaction[col+'_count'] = train_transaction[col].map(col_count)
    test_transaction[col+'_count'] = test_transaction[col].map(col_count)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [5]:
# print(train_transaction[train_transaction['isFraud'] == 1].card1.value_counts())
# print(train_transaction.card1.value_counts())
# print(train_transaction[train_transaction['card1'] == 4504].isFraud.value_counts())
# print(train_transaction[train_transaction['card1'] == 7919].isFraud.value_counts())
# print(train_transaction[train_transaction['card1'] == 9500].isFraud.value_counts())
# print(train_transaction[train_transaction['card1'] == 15885].isFraud.value_counts())
# print(train_transaction.isFraud.value_counts())

# print(train_transaction[['dist1','dist2','addr1','addr2']].head(10))
# print(train_transaction[train_transaction['isFraud'] == 1].head(10))
# print(train_transaction.hour.value_counts())
# print(train_transaction.isFraud.value_counts())



# print(train_transaction.corr())

print(train_transaction[train_transaction['C5'] == 0].isFraud.value_counts())
print(train_transaction[train_transaction['C5'] == 1].isFraud.value_counts())
print(train_transaction[train_transaction['C14'] == 0].isFraud.value_counts())
print(train_transaction[train_transaction['C14'] == 1].isFraud.value_counts())
print(train_transaction.V70.fillna(7).value_counts())
print(train_transaction[train_transaction['V70'] == 0].isFraud.value_counts())
print(train_transaction[train_transaction['V70'].isna()].isFraud.value_counts())

0    353676
1     18759
Name: isFraud, dtype: int64
0    121954
1       929
Name: isFraud, dtype: int64
0    30733
1     5214
Name: isFraud, dtype: int64
0    311589
1      8600
Name: isFraud, dtype: int64
0.0    317801
1.0    184177
7.0     77096
2.0      9739
3.0      1296
4.0       253
5.0       177
6.0         1
Name: V70, dtype: int64
0    303021
1     14780
Name: isFraud, dtype: int64
0    72587
1     4509
Name: isFraud, dtype: int64


In [6]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)
# train = train.merge(train_f2, how='left', left_index=True, right_index=True)
# test = test.merge(test_f2, how='left', left_index=True, right_index=True)
# train = train.merge(train_pred['pred'], how='left', left_index=True, right_index=True)
# test = test.merge(test_pred['pred'], how='left', left_index=True, right_index=True)
train[train['isFraud'] == 1].to_csv('./fraud.csv')
print(train.shape)
print(test.shape)

test['isFraud'] = 0
y_train = train['isFraud'].copy()
y_test = test['isFraud'].copy()
del train_identity, test_identity
# del train_transaction, test_transaction
# del train['card1'],test['card1'],train['card2'],test['card2']
# del train['TransactionDT'],test['TransactionDT']

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.drop('isFraud', axis=1)

# del train, test

# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

(590540, 467)
(506691, 466)


In [7]:
%%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA 
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)

debug = True
if debug:
    split_pos = X_train.shape[0]*4//5
    y_test = y_train.iloc[split_pos:]
    y_train = y_train.iloc[:split_pos]
    X_test = X_train.iloc[split_pos:,:]
    X_train = X_train.iloc[:split_pos,:]

Memory usage of dataframe is 2124.05 MB
Memory usage after optimization is: 619.23 MB
Decreased by 70.8%
Memory usage of dataframe is 1825.30 MB
Memory usage after optimization is: 541.88 MB
Decreased by 70.3%
CPU times: user 2min 28s, sys: 6min 30s, total: 8min 59s
Wall time: 8min 59s


In [8]:
# def encode_vt(train, vali, id_cate, target, use_bayes_smooth):
#     col_name = '_'.join(id_cate) + '_cvr'
#     col_name2 = '_'.join(id_cate) + '_sum'
#     col_name3 = '_'.join(id_cate) + '_size'
#     grouped = train.groupby(id_cate, as_index=False)[target].agg({"C": "size", "V": "sum"})
#     C = grouped['C']
#     V = grouped['V']
#     if (use_bayes_smooth):
#         print('start smooth')
#         hyper = HyperParam(1, 1)
#         hyper.update_from_data(C, V)
#         print('end smooth')
#         grouped[col_name] = (hyper.alpha + V) / (hyper.alpha + hyper.beta + C)
#     else:
# #         grouped[col_name] = ((V / C) + 0.375 * (2.0 - C.map(lambda x:min(x,10.0)) * 0.1))/(1 + (2.0 - C.map(lambda x:min(x,10.0)) * 0.1))
#         grouped[col_name] = (V / C)
#         grouped[col_name2] = V
#         grouped[col_name3] = C
#     grouped[col_name] = grouped[col_name].astype('float32')
#     grouped[col_name2] = grouped[col_name2].astype('float32')
#     grouped[col_name3] = grouped[col_name3].astype('float32')
#     df = vali[id_cate].merge(grouped, 'left', id_cate)[[col_name,col_name2,col_name3]]
#     df = np.asarray(df, dtype=np.float32)
#     del grouped
#     return df

# X_train['isFraud'] = y_train
# X_test['isFraud'] = y_test
# col_14 = [['card1'], ['card2'],['V258']]
# target = 'isFraud'
# use_bayes_smooth = 0
# col_var = []
# vali = X_test
# train = X_train
# for id_cate in col_14:
#     print(id_cate)
#     for id in id_cate:
#         train[id] = train[id].fillna(-1)
#         vali[id] = vali[id].fillna(-1)
#     col_name = '_'.join(id_cate) + '_cvr'
#     col_name2 = '_'.join(id_cate) + '_sum'
#     col_name3 = '_'.join(id_cate) + '_size'
#     col_var.append(col_name)
#     col_var.append(col_name2)
#     col_var.append(col_name3)
#     bayes_feature = encode_vt(train, vali, id_cate, target, use_bayes_smooth)
#     print(bayes_feature)
#     vali[col_name] = 0
#     vali[col_name2] = 0
#     vali[col_name3] = 0
#     vali[[col_name,col_name2,col_name3]] = bayes_feature
#     skf = KFold(n_splits=5, shuffle=False, random_state=15).split(train[target].values)
#     train[col_name] = 0
#     train[col_name2] = 0
#     train[col_name3] = 0

#     for train_idx, test_idx in skf:
#         # print(id_cate, target)
#         train_x = train.iloc[train_idx]
#         test_x = train.iloc[test_idx]
#         bayes_feature = encode_vt(train_x, test_x, id_cate, target, use_bayes_smooth)
#         train.ix[train.iloc[test_idx].index, [col_name,col_name2,col_name3]] = bayes_feature
#         del bayes_feature
        
# del X_train['isFraud'],X_test['isFraud']

In [9]:
%%time

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
folds = 3
kf = KFold(n_splits = folds, shuffle = True, random_state=seed)
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X_train.shape[0])
X_test_pred = pd.DataFrame(index = X_test.index)
X_train_pred = pd.DataFrame(index = X_train.index)
i = 0
for tr_idx, val_idx in kf.split(X_train, y_train):
    i+=1
    clf = xgb.XGBClassifier(
        n_estimators=500,
        max_depth=9,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        tree_method='gpu_hist'
    )
    
    X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    clf.fit(X_tr, y_tr)
    y_pred_train = clf.predict_proba(X_vl)[:,1]
    y_oof[val_idx] = y_pred_train
    print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
    
    y_preds+= clf.predict_proba(X_test)[:,1] / folds
    X_test_pred['pred'+str(i)] = clf.predict_proba(X_test)[:,1]
    if debug:    
        print("debug:",roc_auc_score(y_test, clf.predict_proba(X_test)[:,1] / folds))   

if debug:    
    print("debug:",roc_auc_score(y_test, y_preds))  

X_train_pred['pred'] = y_oof
X_train_pred['isFraud'] = y_train
X_train_pred['card1'] = X_train['card1']
X_train_pred['card2'] = X_train['card2']
X_train_pred['TransactionAmt'] = X_train['TransactionAmt']
X_train_pred.to_csv('simple_xgboost_cv2.csv')
X_test_pred['card1'] = X_test['card1']
X_test_pred['card2'] = X_test['card2']
X_test_pred['TransactionAmt'] = X_test['TransactionAmt']
X_test_pred.to_csv('simple_xgboost_pred2.csv')

ROC AUC 0.9651797879110485
debug: 0.9222116601476789
ROC AUC 0.9660645726520036
debug: 0.9219459369719023
ROC AUC 0.9668999044202873
debug: 0.9214337721426485
debug: 0.9282413070746005
CPU times: user 1min 42s, sys: 57.3 s, total: 2min 39s
Wall time: 2min 39s


In [10]:
# temp = X_train.loc[2987367:2987368]
# temp2 = X_train.loc[2987366:2987367]
# temp11 = train_transaction.loc[2987367:2987368]
# temp22 = train_transaction.loc[2987366:2987367]
# temp3 = temp.copy()
# # print(temp.values)
# # clf.predict_proba(X_train.loc[2990196:2990197])[:,1]
# print(X_train.V70.value_counts())
# for i in range(X_train.shape[1]-1,-1,-1):
    
#     temp3.iloc[0,i] = temp2.iloc[0,i]
#     print(i,X_train.columns[i],temp.iloc[0,i],temp2.iloc[0,i],temp11[X_train.columns[i]].iloc[0],temp22[X_train.columns[i]].iloc[0],clf.predict_proba(temp3)[:,1])
    
# # C5 C14 V70

0.0    251774
1.0    144743
2.0      7687
3.0      1021
4.0       186
5.0       130
6.0         1
Name: V70, dtype: int64


KeyError: 'DeviceInfo'

In [11]:
from sklearn.feature_extraction import DictVectorizer
# print(eli5.explain_prediction_xgboost(clf,X_test.iloc[0,:], vec = vec))
eli5.show_weights(clf)
# print(res.data)
# eli5.explain_prediction(clf,X_test.iloc[10,:])
# eli5.show_prediction(clf, X_train.iloc[10:11,:],show_feature_values=True)


Weight,Feature
0.1281,V258
0.0265,V70
0.0235,V189
0.0175,V91
0.0139,V294
0.0130,C1_count
0.0105,C14
0.0103,V246
0.0100,V244
0.0093,C4


In [12]:
import lightgbm as lgb
# feat_col = [x for x in X_train.columns if x in ['stationID','hour', 'min', 'minandhour','num_neighboor', 'station_time0', 'station_time1',
#        'station_time2', 'station_time3', 'station_pos', 'station_pos2',  'datediff',
#        'station_time_diff_0', 'station_time_diff_1', 'station_time_diff_2',
#        'station_time_diff_3', 'th', 'tl', 'cloud', 'rain1','line0','line0b','line1','line2',
#        'rain2', 'cloud2', 'cloud3', 'sun', 'weekday', 'weekend','type','railway','busstation'] or x.startswith("svd") or x.startswith("w2v") or x.startswith("poi")]
cate = [x for x in X_train.columns if (x == 'ProductCD' or  x.startswith("addr") or x.startswith("card") or 
                                       x.endswith("domain") or x.startswith("Device")) and not x.endswith("count") ]
print(cate)
params = {'application': 'binary',
          'boosting': 'gbdt',
          'metric': 'auc',
          'max_depth': 9,
          'learning_rate': 0.05,
          'bagging_fraction': 0.9,
          'feature_fraction': 0.9,
          'verbosity': -1,
#           'min_child_weight': 30,
          'data_random_seed': 17}
early_stop = 300
verbose_eval = 30
num_rounds = 2000
# 
folds = 3
kf = KFold(n_splits = folds, shuffle = True, random_state=seed)
y_preds2 = np.zeros(X_test.shape[0])
y_oof = np.zeros(X_train.shape[0])
feature_importance_df = pd.DataFrame()
i = 0
for tr_idx, val_idx in kf.split(X_train, y_train):

    
    X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    d_train = lgb.Dataset(X_tr, label=y_tr,categorical_feature = cate)
    d_valid = lgb.Dataset(X_vl, label=y_vl,categorical_feature = cate)
    watchlist = [d_valid]
    if debug:
        d_test = lgb.Dataset(X_test, label=y_test,categorical_feature = cate)
        watchlist.append(d_test)
    
    
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      early_stopping_rounds=early_stop)
        
    y_pred_train = model.predict(X_vl)
    y_oof[val_idx] = y_pred_train
    print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
    
    y_preds2+= model.predict(X_test) / folds
    
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = X_tr.columns
    fold_importance_df["importance"] = model.feature_importance()
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    if debug:    
        print("debug:",roc_auc_score(y_test, model.predict(X_test) / folds))  
    i+=1

if debug:    
    print("debug:",roc_auc_score(y_test, y_preds2))  
    print("debug:",roc_auc_score(y_test, (y_preds + y_preds2)*0.5))  




['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'DeviceType', 'DeviceInfo']


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 300 rounds.
[30]	valid_0's auc: 0.911836	valid_1's auc: 0.88039
[60]	valid_0's auc: 0.931367	valid_1's auc: 0.890919
[90]	valid_0's auc: 0.941492	valid_1's auc: 0.897248
[120]	valid_0's auc: 0.946018	valid_1's auc: 0.900523
[150]	valid_0's auc: 0.949271	valid_1's auc: 0.902305
[180]	valid_0's auc: 0.951874	valid_1's auc: 0.902944
[210]	valid_0's auc: 0.953831	valid_1's auc: 0.904121
[240]	valid_0's auc: 0.955539	valid_1's auc: 0.904966
[270]	valid_0's auc: 0.956691	valid_1's auc: 0.905279
[300]	valid_0's auc: 0.95769	valid_1's auc: 0.905831
[330]	valid_0's auc: 0.958753	valid_1's auc: 0.906763
[360]	valid_0's auc: 0.959565	valid_1's auc: 0.906571
[390]	valid_0's auc: 0.960779	valid_1's auc: 0.907805
[420]	valid_0's auc: 0.961368	valid_1's auc: 0.908186
[450]	valid_0's auc: 0.962064	valid_1's auc: 0.908929
[480]	valid_0's auc: 0.962583	valid_1's auc: 0.909291
[510]	valid_0's auc: 0.963081	valid_1's auc: 0.909603
[540]	valid_0's auc: 0.9

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 300 rounds.
[30]	valid_0's auc: 0.912481	valid_1's auc: 0.877294
[60]	valid_0's auc: 0.928839	valid_1's auc: 0.88768
[90]	valid_0's auc: 0.938475	valid_1's auc: 0.89551
[120]	valid_0's auc: 0.943713	valid_1's auc: 0.8984
[150]	valid_0's auc: 0.947772	valid_1's auc: 0.899594
[180]	valid_0's auc: 0.950437	valid_1's auc: 0.900999
[210]	valid_0's auc: 0.952218	valid_1's auc: 0.902119
[240]	valid_0's auc: 0.954356	valid_1's auc: 0.903973
[270]	valid_0's auc: 0.955699	valid_1's auc: 0.904262
[300]	valid_0's auc: 0.956877	valid_1's auc: 0.905118
[330]	valid_0's auc: 0.957749	valid_1's auc: 0.905572
[360]	valid_0's auc: 0.958696	valid_1's auc: 0.905663
[390]	valid_0's auc: 0.959303	valid_1's auc: 0.905844
[420]	valid_0's auc: 0.960142	valid_1's auc: 0.906169
[450]	valid_0's auc: 0.960784	valid_1's auc: 0.906342
[480]	valid_0's auc: 0.961717	valid_1's auc: 0.906749
[510]	valid_0's auc: 0.962096	valid_1's auc: 0.906903
[540]	valid_0's auc: 0.962

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 300 rounds.
[30]	valid_0's auc: 0.914319	valid_1's auc: 0.877362
[60]	valid_0's auc: 0.933863	valid_1's auc: 0.890319
[90]	valid_0's auc: 0.943173	valid_1's auc: 0.897029
[120]	valid_0's auc: 0.948974	valid_1's auc: 0.899623
[150]	valid_0's auc: 0.952632	valid_1's auc: 0.902621
[180]	valid_0's auc: 0.955467	valid_1's auc: 0.904104
[210]	valid_0's auc: 0.957317	valid_1's auc: 0.906205
[240]	valid_0's auc: 0.959194	valid_1's auc: 0.90745
[270]	valid_0's auc: 0.960485	valid_1's auc: 0.907848
[300]	valid_0's auc: 0.961713	valid_1's auc: 0.908161
[330]	valid_0's auc: 0.962611	valid_1's auc: 0.908649
[360]	valid_0's auc: 0.963573	valid_1's auc: 0.909005
[390]	valid_0's auc: 0.964518	valid_1's auc: 0.910257
[420]	valid_0's auc: 0.965012	valid_1's auc: 0.910646
[450]	valid_0's auc: 0.965633	valid_1's auc: 0.911542
[480]	valid_0's auc: 0.966044	valid_1's auc: 0.911816
[510]	valid_0's auc: 0.966793	valid_1's auc: 0.912497
[540]	valid_0's auc: 0.

In [13]:
if debug:    
    print("debug:",roc_auc_score(y_test, y_preds))  
    print("debug:",roc_auc_score(y_test, y_preds2))  
    print("debug:",roc_auc_score(y_test, (y_preds + y_preds2)*0.5)) 

debug: 0.9282413070746005
debug: 0.9213236971218736
debug: 0.9344166307409463


In [ ]:
# eli5.show_prediction(model,X_test.iloc[1,:])
# eli5.show_weights(model)

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:100].index)
print(cols)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="Feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [ ]:

sample_submission['isFraud'] = (y_preds + y_preds2)*0.5
sample_submission.to_csv('simple_ensemble.csv')
